In [1]:
################### CPU 이용시 설치 ###################
# 1. 기존 제거
!pip uninstall -y torch torchvision torchaudio torch-geometric torch-scatter torch-sparse

# 2. PyTorch 2.4.1 (CPU 전용) 설치
# --index-url을 지정하여 CPU 전용 가벼운 바이너리를 받습니다.
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cpu

# 3. PyG 의존성 설치 (CPU용 Wheel)
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.4.0+cpu.html

# 4. PyG 메인 설치
!pip install torch-geometric
import torch
print(f"Torch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.9/542.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.6 MB/s eta 

In [30]:
import torch
from torch_geometric.nn import TGNMemory
from torch_geometric.nn.models.tgn import IdentityMessage, LastAggregator

print(">>> TGNMemory 모듈 테스트 시작\n")

num_nodes = 5        # 전체 노드 5개 (0~4)
memory_dim = 4       # 메모리 차원 4
time_dim = 4         # 시간 임베딩 차원 4
raw_msg_dim = 4      # 메시지 차원 4

memory_module = TGNMemory(
    num_nodes=num_nodes,
    raw_msg_dim=raw_msg_dim,
    memory_dim=memory_dim,
    time_dim=time_dim,
    message_module=IdentityMessage(raw_msg_dim, memory_dim, time_dim),
    aggregator_module=LastAggregator()
)

print("[Step 1] 초기화 확인")

node_0_mem = memory_module.memory
memory_module.memory

>>> TGNMemory 모듈 테스트 시작

[Step 1] 초기화 확인


tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [33]:
memory_module.last_update.dtype

torch.int64

In [2]:
()

# ---------------------------------------------------------
# 검증 2: 메모리 업데이트 (Interaction 발생)
# ---------------------------------------------------------
print("[Step 2] 이벤트 발생 및 메모리 업데이트")

# 가상 데이터: 0번 노드가 1번 노드와 상호작용함
src = torch.tensor([0])
dst = torch.tensor([1])
t = torch.tensor([1.0])  # 시간 (float)
msg = torch.randn(1, raw_msg_dim) # 랜덤 메시지 생성

# 업데이트 실행 (내부적으로 메시지 생성 -> 집계 -> GRU 업데이트 수행)
memory_module.update_state(src, dst, t, msg)

print(" >> update_state() 실행 완료 (0번, 1번 노드 업데이트 됨)")
print("-" * 30)

# ---------------------------------------------------------
# 검증 3: 업데이트 결과 확인
# ---------------------------------------------------------
print("[Step 3] 업데이트 후 값 변화 확인")

# 0번 노드 (업데이트 대상): 값이 변해야 함
new_node_0 = memory_module.memory(torch.tensor([0]))
print(f" - 0번 노드 (Updated): {new_node_0.tolist()}")
# 2번 노드 (업데이트 안 됨): 여전히 0이어야 함
node_2 = memory_module.memory(torch.tensor([2]))
print(f" - 2번 노드 (No Event): {node_2.tolist()}")

# 데이터 타입 재확인
print(f" - 업데이트 후 dtype: {new_node_0.dtype}")
print("-" * 30)
# ---------------------------------------------------------
# 검증 4: Reset 및 Detach
# ---------------------------------------------------------
print("[Step 4] 유틸리티 기능 확인")

# Detach (BPTT 끊기)
memory_module.detach()
print(" - Detach 완료 (Gradient 끊김)")
# Reset (초기화)
memory_module.reset_state()
reset_node_0 = memory_module.memory(torch.tensor([0]))

# 다시 0으로 돌아갔는지 확인 (합이 0이어야 함)
is_zero = torch.sum(reset_node_0) == 0
print(f" - Reset 완료 여부: {is_zero.item()}")

>>> TGNMemory 모듈 테스트 시작

[Step 1] 초기화 확인


TypeError: 'Tensor' object is not callable

In [ ]:
import torch
from torch_geometric.nn import TransformerConv

# --- 1. 모듈 준비 ---
# 실제 모델과 똑같은 설정으로 만듭니다.
# 예: 입력 128차원 -> 출력 128차원
test_gnn = TransformerConv(in_channels=128, out_channels=128)

# 장치 설정 (GPU 있다면 GPU로)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_gnn = test_gnn.to(device)  # 모듈을 장치로 이동

# --- 2. 가짜 데이터(Dummy Data) 생성 ---
# 배치 사이즈가 10이고, 차원이 128인 데이터를 가정
# [중요] 여기서 .float()를 안 하면 방금 겪은 에러가 또 납니다!
dummy_input = torch.randn(10, 128).float().to(device)

# 엣지 인덱스 (테스트용으로 비어있는 엣지 생성)
# [중요] 인덱스는 무조건 .long() 이어야 함
dummy_edge = torch.zeros((2, 0), dtype=torch.long).to(device)

# --- 3. 실행 및 확인 ---
test_gnn.eval()  # 평가 모드 (Dropout 끄기)

with torch.no_grad():  # 테스트니까 미분 계산 끔
    output = test_gnn(dummy_input, dummy_edge)

print(f"입력 크기: {dummy_input.shape}")
print(f"출력 크기: {output.shape}")
# 결과가 torch.Size([10, 128]) 처럼 나오면 정상 작동!